This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [2]:
import pandas as pd
import numpy as np
from random import randint

data = pd.read_csv('data/data_000637.txt')
df = pd.DataFrame(data)
N = randint(10000, 1310720)
print('Checking the documentation:\n',df)
print('Number of rows printed: ', N)

Checking the documentation:
          HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0           1     0          123  3869200167        2374        26
1           1     0          124  3869200167        2374        27
2           1     0           63  3869200167        2553        28
3           1     0           64  3869200167        2558        19
4           1     0           64  3869200167        2760        25
...       ...   ...          ...         ...         ...       ...
1310715     1     0           62  3869211171         762        14
1310716     1     1            4  3869211171         763        11
1310717     1     0           64  3869211171         764         0
1310718     1     0          139  3869211171         769         0
1310719     1     0           61  3869211171         762        18

[1310720 rows x 6 columns]
Number of rows printed:  939233


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [3]:
df_cntbx = df.loc[:, ['ORBIT_CNT', 'BX_COUNTER']]
df_cnt = df.loc[:, ['ORBIT_CNT']]
df_bx = df.loc[:, ['BX_COUNTER']]

#Commented algorithm because of issues understanding question. This selects 1st and last BX_COUNTER
#of an ORBIT_CNT and subtracts one another in order to give all BX_COUNTERS in the interval.
'''
save_position = 0
save_number_bxcounter = 0
n_orbits = 0
nrows = len(df)
for i in range(nrows):
    orbit_cnt_int1 = int(df_cnt.iloc[i])
    orbit_cnt_int2 = int(df_cnt.iloc[i+1])
    if orbit_cnt_int1 != orbit_cnt_int2:
        orbit_bx_last = int(df_bx.iloc[i])
        orbit_bx_first = int(df_bx.iloc[save_position])
        sub = orbit_bx_last-orbit_bx_first
        save_number_bxcounter += sub
        n_orbits += 1
        #print('ORBIT_CNT value:', orbit_cnt_int1, '\nBX_COUNTER estimation:', sub, '\n')
        save_position = i+1
print('BX_COUNTER estimation in 1 orbit:', float(save_number_bxcounter/n_orbits))
'''

#Algorithm which I consider to be the right EX question where I simply sum all BX_COUNTERS inside an ORBIT_CNT.
df_cntbx = df_cntbx.groupby(['ORBIT_CNT'], as_index=False)['BX_COUNTER'].sum()
print('Number of BX in a ORBIT:\n', df_cntbx)

Number of BX in a ORBIT:
         ORBIT_CNT  BX_COUNTER
0      3869200167      124133
1      3869200168       97201
2      3869200169      144343
3      3869200170      217462
4      3869200171      228822
...           ...         ...
10996  3869211167      361697
10997  3869211168      205302
10998  3869211169      311322
10999  3869211170      277743
11000  3869211171        7573

[11001 rows x 2 columns]


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [4]:
import datetime

X = 3563

n_row = 0
row_init = df.iloc[n_row]
ns_init = row_init['TDC_MEAS']*25/30 + row_init['BX_COUNTER']*25 + row_init['ORBIT_CNT']*X*25
                                                   
s_init = ns_init *10**-9
                                                   
print(row_init)
print("Duration of the data taking of N sample:")
print(str(datetime.timedelta(seconds = s_init)))

HEAD                    1
FPGA                    0
TDC_CHANNEL           123
ORBIT_CNT      3869200167
BX_COUNTER           2374
TDC_MEAS               26
Name: 0, dtype: int64
Duration of the data taking of N sample:
3 days, 23:44:09.004935


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [5]:
X = 3563
df['Row Time'] =  (df['TDC_MEAS']*25/30 + df['BX_COUNTER']*25 + df['ORBIT_CNT']*X*25).cumsum()
print('Dataframe with new column with the absolute time in ns:\n', df)

Dataframe with new column with the absolute time in ns:
          HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS  \
0           1     0          123  3869200167        2374        26   
1           1     0          124  3869200167        2374        27   
2           1     0           63  3869200167        2553        28   
3           1     0           64  3869200167        2558        19   
4           1     0           64  3869200167        2760        25   
...       ...   ...          ...         ...         ...       ...   
1310715     1     0           62  3869211171         762        14   
1310716     1     1            4  3869211171         763        11   
1310717     1     0           64  3869211171         764         0   
1310718     1     0          139  3869211171         769         0   
1310719     1     0           61  3869211171         762        18   

             Row Time  
0        3.446490e+14  
1        6.892980e+14  
2        1.033947e+15  
3     

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [6]:
df_maxTDC = df.groupby(['TDC_MEAS'], as_index=False).max()
print('TDC channels with most counts:\n', df_maxTDC.iloc[-3:])

TDC channels with most counts:
     TDC_MEAS  HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER      Row Time
27        27     1     1          130  3869211170        3563  4.517235e+20
28        28     1     1          130  3869211171        3563  4.517322e+20
29        29     1     1          130  3869211171        3563  4.517349e+20


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [7]:
df1 = df[df.isna().any(axis=1)]
print('Number of non-empty orbits: ', df1.shape[0])

df_ORBIT = df.groupby(['ORBIT_CNT'], as_index=False)['TDC_CHANNEL'].max()
print('Number of ORBITS_CNT with TDC_CHANNEL=139: ', df_ORBIT.shape[0])

Number of non-empty orbits:  0
Number of ORBITS_CNT with TDC_CHANNEL=139:  11001


7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel